In [ ]:
import pandas as pd
import numpy as np
import re
import hashlib
from datetime import datetime, time
from pathlib import Path

src = "data_s2_2025.xlsx"

df = pd.read_excel("data_s2_2025.xlsx", skiprows=2)



In [8]:
df.columns = [re.sub(r"\s+", "_", c.strip().lower()) for c in df.columns]

In [9]:
df.head()
print(df.columns)     # see available columns

Index(['unit_of_study_code', 'unit_of_study_name', 'session',
       'anticipated_enrolments', 'actual_enrolments', 'allocation_status',
       'error_text', 'activity_type', 'activity_description', 'activity_name',
       'activity_date', 'activity_start', 'activity_end', 'paycode',
       'teaching_role', 'staffid', 'name', 'faculty', 'school', 'department',
       'units_(hrs)'],
      dtype='object')


In [11]:
for col in df.columns:
    print(f"\n{col}:")
    print(df[col].dropna().unique())


unit_of_study_code:
['COMP2022' 'COMP2123' 'COMP2922' 'COMP3419' 'COMP3520' 'COMP3888'
 'COMP3988' 'COMP4415' 'COMP5047' 'COMP5048' 'COMP5216' 'COMP5310'
 'COMP5318' 'COMP5328' 'COMP5338' 'COMP5339' 'COMP5348' 'COMP5415'
 'COMP5416' 'COMP5530' 'COMP5615' 'COMP5618' 'COMP5703' 'COMP9001'
 'COMP9003' 'COMP9120' 'COMP9121' 'COMP9123' 'COMP9201' 'COMP9208'
 'COMP9412' 'CSEC3616' 'CSEC3888' 'CSEC5614' 'CSEC5616' 'CSYS5010'
 'CSYS5030' 'CSYS5040' 'DATA1002' 'DATA1902' 'DATA3406' 'DATA5207'
 'GEN3107S' 'HTIN5003' 'HTIN5005' 'HTIN6011' 'INFO1110' 'INFO1112'
 'INFO1113' 'INFO1910' 'INFO3315' 'INFO3333' 'INFO4001' 'INFO4002'
 'INFO5060' 'INFO5301' 'INFO5306' 'INFO5990' 'INFO5991' 'INFO5992'
 'INFO5993' 'INFO6007' 'ISYS2120' 'ISYS3888' 'OCIS6022' 'OCMP5048'
 'OCMP5310' 'OCMP5318' 'OCMP5328' 'OCMP5329' 'OCMP5338' 'OCMP5339'
 'OCMP5349' 'OCMP5426' 'OCMP5617' 'OCMP5618' 'OCSE5614' 'OCSE5615'
 'OCSE5616' 'OCSE5619' 'OCSE5707' 'OCSE5708' 'ODAT5013' 'ODAT5707'
 'ODAT5708' 'OINF5301' 'OINF5990' 'OINF59

Course / Offering Level

unit_of_study_code → unit_of_study_name
👉 Ensure each code consistently maps to one name.

unit_of_study_code → session
👉 What sessions each unit runs in.

unit_of_study_code → anticipated_enrolments vs actual_enrolments
👉 Spot gaps, useful for seeding budget.


Activity Level

unit_of_study_code → activity_type
👉 Which activity types belong to each unit.

activity_type → activity_description
👉 See if descriptions are consistent per type.

activity_type → paycode
👉 Map activity types to paycodes (feeding payrate).

activity_name → activity_date / activity_start / activity_end
👉 Distinguish pattern activities vs session occurrences.

Staff & Roles

name → teaching_role
👉 Who is a Tutor vs UC vs other roles.

name → unit_of_study_code
👉 Which units each staff teaches.

teaching_role → paycode
👉 Useful if pay differs by role.


unit_of_study_code → faculty / school / department
👉 Check consistency across units; seed them if your schema later needs org metadata.

In [13]:
# Units to names (check consistency)
df.groupby("unit_of_study_code")["unit_of_study_name"].unique()

# Sessions per unit
df.groupby("unit_of_study_code")["session"].unique()

# Activity types per unit
df.groupby("unit_of_study_code")["activity_type"].unique()

# Paycodes per activity type
df.groupby("activity_type")["paycode"].unique()

# Staff roles
df.groupby("name")["teaching_role"].unique()

# Activities to schedule
df.groupby("activity_name")[["activity_date","activity_start","activity_end"]].agg(lambda x: x.unique())


,activity_date,activity_start,activity_end
activity_name,,,
COMP2022-S2C-ND-CC/BootCamp-IGN/01,"[2025-07-21 00:00:00, 2025-07-23 00:00:00]",[10:00:00],[16:00:00]
COMP2022-S2C-ND-CC/BootCamp-IGN/02,[2025-07-25 00:00:00],[10:00:00],[19:00:00]
COMP2022-S2C-ND-CC/BootCamp-IGN/03,"[2025-07-28 00:00:00, 2025-07-29 00:00:00]",[10:00:00],[12:00:00]
COMP2022-S2C-ND-CC/Lecture/01,"[2025-08-05 00:00:00, 2025-08-12 00:00:00, 202...",[11:00:00],[13:00:00]
COMP2022-S2C-ND-CC/Lecture/02,"[2025-08-05 00:00:00, 2025-08-12 00:00:00, 202...",[16:00:00],[18:00:00]
...,...,...,...
SOFT3888-S2C-ND-CC/Workshop/07,"[2025-08-14 00:00:00, 2025-08-21 00:00:00, 202...",[10:00:00],[12:00:00]
SOFT3888-S2C-ND-CC/Workshop/08,"[2025-08-14 00:00:00, 2025-08-21 00:00:00, 202...",[08:00:00],[10:00:00]
SOFT3888-S2C-ND-CC/Workshop/09,"[2025-08-12 00:00:00, 2025-08-19 00:00:00, 202...",[12:00:00],[14:00:00]
